In [2]:
import numpy as np

In [18]:
class Dense_Layer:
    def __init__(self, n_features, n_neurons):      # layer initialization
        self.weights = np.random.randn(n_neurons, n_features).T     # transposed weight matrix
        self.bias = np.zeros((1, n_neurons))    # bias matrix

    def forward(self, input):       # defining forward pass
        self.input = input
        self.output = np.dot(input, self.weights) + self.bias     # wx + b

    def backward(self, received_grad):      # defining backward pass
        self.dweights = np.dot(self.input.T, received_grad)     # derivative of loss funtion w.r.t weights of the current layer = derivative of output of current layer w.r.t weights (which is input) * derivative of loss function w.r.t output of current layer (received_grad)
        self.dbias = np.sum(received_grad, axis=0, keepdims=True)     # same as above, derivative of output of current layer w.r.t biases is just a vector of ones and its dot product with the received_grad would just sum up all its elements
        self.dinput = np.dot(received_grad, self.weights.T)     # derivative of output of current layer w.r.t input is weights

Explaination of backward pass:

Each row of received_grad would represent one sample. It would contain derivative of loss for that sample with respect to all the outputs of the current layer. Thus, the $j_{th}$ column represents derivative with respect to the $j_{th}$ output. $i_{th}$ row of the transposed input matrix would represent derivative of output of this layer with respect to the $i_{th}$ weight of each neuron. Their dot product thus sums over the gradient of loss of all the samples with respect to the weight of the $j_{th}$ neuron and $i_{th}$ input. The size of resulting matriz is thus same as the weights matrix.

For dbias, since derivative of individual output with respect to its bias is 1, we just need to sum up gradients with respect to the output over all samples to get gradient with respect to that particular bias.

For dinput, $j_{th}$ column of transposed weight matrix is derivative of output of the neuron with respect to $j_{th}$ input feature and as mentioned, $i_{th}$ row of received_grad is the derivtive of $i_{th}$ sample loss with respect to all the outputs. The dot product thus give gradient of $i_{th}$ sample loss with respect to $j_{th}$ feature. The resulting matrix thus has the same shape as input.

In [22]:
class ReLU:
    def forward(self, input):
        self.input = input
        self.output = np.maximum(0, input) # the rectified linear unit activation function

    def backward(self, received_grad):
        self.dinput = received_grad.copy()
        self.dinput[self.input <= 0] = 0   # derivative of output of ReLU w.r.t its input is 1 when the input is positive and 0 otherwise, its multiplication with received_grad would just change the non-positive values to zero and leave other values as it is.



Now we would implement some methods that would help update the weights and biases in order to get a model where the loss is minimized. These are known as optimizers. Below is the implementation of Stochastic Gradient Descent, with or without momentum. Momentum works as follows:
- $\mathbf{m} = \beta\mathbf{m} - \eta\nabla_\theta\mathbf{C(\theta)}$
- $\theta = \theta + \mathbf{m}$

Here, $\theta$ is the parameter vector, $\nabla_\theta\mathbf{C(\theta)}$ is the gradient vector, $\eta$ is the learning rate, m is the parameter momentum vector and $\beta$, know as momentum, is a hyperparameter that shows the decay of the exponentially decaying sum of gradients.

In [9]:
class SGD:
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre(self):
        if self.decay:      # decay of the learning rate
            self.current_learning_rate = self.learning_rate * (1./(1. + self.decay + self.iterations))

    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):      # if parameter momentums are not initialized
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            weight_update = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_update

            bias_update = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbias
            layer.bias_momentums = bias_update

        else:
            weight_update = -self.current_learning_rate * layer.dweights
            bias_update = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_update
        layer.bias += bias_update

    def post(self):
        self.iterations += 1



Below we create a class to implement the RMSProp (Root Mean Square Propagation) Optimizer, which works as per the following two equations:

- $\mathbf{s} = \rho\mathbf{s} + (1 - \rho)\left\{ \nabla_\theta\mathbf{C(\theta)} \circ \nabla_\theta\mathbf{C(\theta)} \right\}$
- $\mathbf{\theta} = \theta - \eta\left\{ \nabla_\theta\mathbf{C(\theta)} \oslash \sqrt{\mathbf{s} + \epsilon} \right\}$

Here, $\circ$ denotes element-wise multiplication and $\oslash$ denotes element-wise division, $\rho$ denots the decay of the exponentially decaying average of squares of gradients and s (known as cache) accumulates this average. $\epsilon$ is a small value to prevent division by zero.

In [10]:
class RMSProp:
    def __init__(self, rho=0.9, eta=0.001, epsilon=1e-7, decay=0):
        self.learning_rate = eta
        self.current_learning_rate = eta
        self.rho = rho
        self.epsilon = epsilon
        self.iterations = 0
        self.decay = decay

    def pre(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1./(1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.bias)

        layer.weight_cache = self.rho * layer.weight_cache + (1-self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + (1-self.rho) * layer.dbias**2

        layer.weights += -self.current_learning_rate * {layer.dweights / np.sqrt(layer.weight_cache + self.epsilon)}
        layer.bias += -self.current_learning_rate * {layer.dbias / np.sqrt(layer.bias_cache + self.epsilon)}

    def post(self):
        self.iterations += 1

Our final Optimizer, known as Adam, combines both the momentum method and RMSProp. It works as follows:

- $\mathbf{m} = \beta_1\mathbf{m} + (1 - \beta_1)\nabla_\theta\mathbf{C(\theta)}$
- $\mathbf{s} = \beta_2\mathbf{s} + (1 - \beta_2)\left\{ \nabla_\theta\mathbf{C(\theta)} \circ \nabla_\theta\mathbf{C(\theta)} \right\}$
- $\mathbf{\hat{m}} = \frac{\mathbf{m}}{1 - \beta_1^t}$
- $\mathbf{\hat{s}} = \frac{\mathbf{s}}{1 - \beta_2^t}$
- $\theta = \theta - \eta\left\{ \mathbf{\hat{m}} \oslash \sqrt{\mathbf{\hat{s}} + \epsilon} \right\}$



In [24]:
class Adam:
    def __init__(self, eta=0.001, decay=0, epsilon=1e-7, beta1=0.9, beta2=0.999):
        self.learning_rate = eta
        self.current_learning_rate = eta
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta1 = beta1
        self.beta2 = beta2

    def pre(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentum = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentum = np.zeros_like(layer.bias)
            layer.bias_cache = np.zeros_like(layer.bias)

        layer.weight_momentum = self.beta1 * layer.weight_momentum + (1-self.beta1) * layer.dweights
        layer.bias_momentum = self.beta1 * layer.bias_momentum + (1-self.beta1) * layer.dbias

        weight_momentum_corrected = layer.weight_momentum / (1 - self.beta1**(self.iterations + 1))
        bias_momentum_corrected = layer.bias_momentum / (1 - self.beta1**(self.iterations + 1))

        layer.weight_cache = self.beta2 * layer.weight_cache + (1-self.beta2) * layer.dweights**2
        layer.bias_cache = self.beta2 * layer.bias_cache + (1-self.beta2) * layer.dbias**2

        weight_cache_corrected = layer.weight_cache / (1 - self.beta2**(self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta2**(self.iterations + 1))

        layer.weights += -self.current_learning_rate * weight_momentum_corrected / (np.sqrt(weight_cache_corrected + self.epsilon))
        layer.bias += -self.current_learning_rate * bias_momentum_corrected / (np.sqrt(bias_cache_corrected + self.epsilon))

    def post(self):
        self.iterations += 1

Now we implement the SoftMAX Activation funtion that is applied to the final layer of the network. It is defined as:
$
S_{i,j} = \frac{e^{z_{i,j}}}{\sum_{l=1}^{L} e^{z_{i,l}}}
$\
Here, $z_{i,j}$ are the outputs of the final layer, i denotes sample number, j denotes the class number (rather, the neuron number of the final layer) and $S_{i,j}$ is the output of the activation function. Instead of $z_{i,j}$, we often use $z_{i,j} - max(z_{i,j})$ for a given sample, in order to comput the exponential.

The backward method uses the following relation:
$$
\frac{\partial S_{i,j}}{\partial z_{i,k}} = S_{i,j} \cdot (\delta_{j,k} - S_{i,k}) = S_{i,j} \delta_{j,k} - S_{i,j} S_{i,k}
$$

Here, $\delta_{j,k}$ is the Kronecker Delta function which is equal to 1 when j = k and 0 otherwise.


In [12]:
class SoftMAX:
    def forward(self, input):
        self.input = input
        exp_values = np.exp(input - np.max(input, axis = 1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, received_grad):
        self.dinputs = np.empty_like(received_grad)

        for index , (single_output, single_dvalue) in enumerate(zip(self.output, received_grad)):   # we take the values of one sample at a time at evaluate the gradient for their loss.
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)   # diagflat would convert the single_output into a diagonal matrix with the single_output as the diagonal, and its multiplication with its transpose produces all the S_{i,j} * S_{i,k}.
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalue)    # multiplying with the received gradient as per chain rule

Explaination of jacobian matrix and chain rule used above:

For a single sample, we get a 2D gradient of softmax output with respect to the input. $Element_{i,j}$ of this matrix is the gradient of $j_{th}$ output with respect to $i_{th}$ input. Thus, in the matrix multiplication of the last step, we sum up gradients with respect to the $i_{th}$ input after mulltiplying them with the respective gradients with respect to the output. This gives a scalar value that becomes the $i_{th}$ element of dinputs[index]. 

The following class computes loss value based on the predicted and true values. The actual loss value is: $
L_i = -\log(\hat{y}_{i,k})
$
Here, $\hat{y}_{i,k}$ is the predicted probablity of the sample belonging to the kth class, where k is the true class.

The backward method uses the following relation to calculate the gradients: 
$$
\frac{\partial L_i}{\partial \hat{y}_{i,j}} = -\frac{y_{i,j}}{\hat{y}_{i,j}}
$$



In [13]:
class Categorical_Cross_EntropyLoss:
    def forward(self, y_pred, y_true):
        n_samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)  # clipped so as to prevent division by zero

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(n_samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self, y_pred, y_true):     # since loss will be the first one to calculate a gradient during backpropagation, it wont receive any gradient (as do the other backward functions), insted it would start with the predicted values
        n_samples = len(y_pred)
        n_labels = len(y_pred[0])

        if len(y_true.shape) == 1:      # converting to one-hot encoded if the provided one is sparse
            y_true = np.eye(n_labels)[y_true]

        self.dinputs = -y_true / y_pred
        self.dinputs = self.dinputs / n_samples
    


The last phase of a typical neural network is : Final Layer -> SoftMAX Activation -> Error Calculation\
During backpropagation, instead of first calculating the gradient of Error with respect to SoftMAX output and then calculating gradient of SoftMAX output with respect to its input and multiplying the two as per chain rule, it is more efficient to directly calculate the gradient of Error with respect to SoftMAX input (output of the final layer). The following class implements this using the following relation:
$$
\frac{\partial L_i}{\partial z_{i,k}} = \hat{y}_{i,k} - y_{i,k}
$$



In [14]:
class Combined_CCELoss_grad_wrt_SoftMAX_input:
    def __init__(self):
        self.activation = SoftMAX()
        self.loss = Categorical_Cross_EntropyLoss()

    def forward(self, input, y_true):   # input here will be the input of the softmax activation function
        self.activation.forward(input)
        self.output = self.activation.output

        return self.loss.forward(self.output, y_true)
    
    def backward(self, y_pred, y_true):
        n_samples = len(y_pred)

        if len(y_true.shape) == 2:      # converting to sparse if the provided one is one-hot encoded
            y_true = np.argmax(y_true, axis=1)      # this gives us the true classes of the samples

        self.dinputs = y_pred.copy()
        self.dinputs[range(n_samples), y_true] -= 1     # y_{i,j} will be one for all the true classes and zero otherwise
        self.dinputs = self.dinputs / n_samples



In [ ]:
# an example implementation

import nnfs
from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt 
nnfs.init()

In [ ]:
x, y = spiral_data(samples=100, classes=3)

dense1 = Dense_Layer(2, 64)
activation1 = ReLU()
dense2 = Dense_Layer(64, 3)
activation_loss = Combined_CCELoss_grad_wrt_SoftMAX_input()

optimizer = Adam()

for epoch in range(10001):
    dense1.forward(x)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = activation_loss.forward(dense2.output, y)

    predictions = np.argmax(activation_loss.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch : {epoch}, ' +
              f'acc : {accuracy},')
        
    activation_loss.backward(activation_loss.output, y)
    dense2.backward(activation_loss.dinputs)
    activation1.backward(dense2.dinput)
    dense1.backward(activation1.dinput)

    optimizer.pre()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post()



epoch : 0, acc : 0.3433333333333333,
epoch : 100, acc : 0.44333333333333336,
epoch : 200, acc : 0.43666666666666665,
epoch : 300, acc : 0.47,
epoch : 400, acc : 0.48333333333333334,
epoch : 500, acc : 0.4766666666666667,
epoch : 600, acc : 0.49,
epoch : 700, acc : 0.49,
epoch : 800, acc : 0.5133333333333333,
epoch : 900, acc : 0.53,
epoch : 1000, acc : 0.5333333333333333,
epoch : 1100, acc : 0.53,
epoch : 1200, acc : 0.5366666666666666,
epoch : 1300, acc : 0.55,
epoch : 1400, acc : 0.5433333333333333,
epoch : 1500, acc : 0.55,
epoch : 1600, acc : 0.5666666666666667,
epoch : 1700, acc : 0.56,
epoch : 1800, acc : 0.57,
epoch : 1900, acc : 0.5833333333333334,
epoch : 2000, acc : 0.5933333333333334,
epoch : 2100, acc : 0.5966666666666667,
epoch : 2200, acc : 0.6166666666666667,
epoch : 2300, acc : 0.6166666666666667,
epoch : 2400, acc : 0.6233333333333333,
epoch : 2500, acc : 0.6366666666666667,
epoch : 2600, acc : 0.64,
epoch : 2700, acc : 0.6366666666666667,
epoch : 2800, acc : 0.6433333